In [46]:
# Cell 1: 기본 설정 & import

from pathlib import Path
from typing import List, Tuple, Dict
import re
import sys

import numpy as np
import torch
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

# ---------- 사용자 설정 ----------
IMG_ROOT = Path("./dataset/deit_base_16_imagenet/SMI_iter/W8A8").expanduser().resolve()

# 결과 저장 폴더
SAVE_DIR = Path("./observation/iter_noise").expanduser().resolve()

# 고주파/저주파 cutoff 비율 (0~1)
# 1에 가까울수록 high frequency 영역이 좁아짐
CUTOFF_RATIO = 0.8

SAVE_DIR.mkdir(parents=True, exist_ok=True)

print(f"[INFO] IMG_ROOT     : {IMG_ROOT}")
print(f"[INFO] SAVE_DIR     : {SAVE_DIR}")
print(f"[INFO] CUTOFF_RATIO : {CUTOFF_RATIO}")

[INFO] IMG_ROOT     : /home/jener05458/src/EdgeMI/TBD_MI/dataset/deit_base_16_imagenet/SMI_iter/W8A8
[INFO] SAVE_DIR     : /home/jener05458/src/EdgeMI/TBD_MI/observation/iter_noise
[INFO] CUTOFF_RATIO : 0.8


In [47]:
# Cell 2: 이미지 찾기 (IMG_ROOT 아래에서 DMI-iter / SMI-iter 폴더만 대상)

# 예: "DMI-50-0-32-W4A8", "SMI-100-[50,100,200,300]-[...]-0-32-W4A8"
_iter_root_pattern = re.compile(r"^(DMI|SMI)-\d+-")

def find_images(img_root: Path) -> List[Path]:
    """
    img_root (예: .../DMI/W4A8)을 입력받아,
    그 내부의 DMI-{iter}-..., SMI-{iter}-... 폴더들에서 *.png 이미지를 모두 수집한다.
    """
    if not img_root.exists():
        raise FileNotFoundError(f"[ERROR] 이미지 폴더가 존재하지 않습니다: {img_root}")

    image_paths: List[Path] = []

    # 1) 하위 디렉토리 중 DMI-*/SMI-* 폴더만 선택
    for subdir in sorted(img_root.iterdir()):
        if not subdir.is_dir():
            continue

        name = subdir.name
        if _iter_root_pattern.match(name) is None:
            # iteration 폴더가 아닌 경우 skip
            continue

        # 2) 해당 폴더에서 모든 .jpg 파일 찾기
        jpgs = sorted(p for p in subdir.rglob("*.png") if p.is_file())
        image_paths.extend(jpgs)

    if len(image_paths) == 0:
        raise FileNotFoundError(
            f"[ERROR] '{img_root}' 아래에서 png 이미지를 찾지 못했습니다. "
            "DMI-iter 또는 SMI-iter 폴더 구조를 확인하세요."
        )
    
    return image_paths


# 실행
all_image_paths = find_images(IMG_ROOT)
print(f"[INFO] Found {len(all_image_paths)} images")
all_image_paths[:5]


[INFO] Found 1280 images


[PosixPath('/home/jener05458/src/EdgeMI/TBD_MI/dataset/deit_base_16_imagenet/SMI_iter/W8A8/SMI-100-[50, 100, 200, 300]-[0.3, 0.3, 0.3, 0.3]-0-32-W8A8/0-0.png'),
 PosixPath('/home/jener05458/src/EdgeMI/TBD_MI/dataset/deit_base_16_imagenet/SMI_iter/W8A8/SMI-100-[50, 100, 200, 300]-[0.3, 0.3, 0.3, 0.3]-0-32-W8A8/0-1.png'),
 PosixPath('/home/jener05458/src/EdgeMI/TBD_MI/dataset/deit_base_16_imagenet/SMI_iter/W8A8/SMI-100-[50, 100, 200, 300]-[0.3, 0.3, 0.3, 0.3]-0-32-W8A8/0-10.png'),
 PosixPath('/home/jener05458/src/EdgeMI/TBD_MI/dataset/deit_base_16_imagenet/SMI_iter/W8A8/SMI-100-[50, 100, 200, 300]-[0.3, 0.3, 0.3, 0.3]-0-32-W8A8/0-11.png'),
 PosixPath('/home/jener05458/src/EdgeMI/TBD_MI/dataset/deit_base_16_imagenet/SMI_iter/W8A8/SMI-100-[50, 100, 200, 300]-[0.3, 0.3, 0.3, 0.3]-0-32-W8A8/0-12.png')]

In [48]:
# Cell 3: 이미지 로딩 (torch.Tensor [C,H,W], float32 in [0,1])

from torchvision import transforms

def load_images(image_paths: List[Path]) -> List[Tuple[Path, torch.Tensor]]:
    """
    image_paths 리스트를 순회하며 이미지를 로드하여 텐서로 변환한다.
    반환 형식: [(Path, Tensor[C,H,W]), ...]
    """
    to_tensor = transforms.ToTensor()
    loaded: List[Tuple[Path, torch.Tensor]] = []

    for p in image_paths:
        try:
            img = Image.open(p).convert("RGB")
            tensor = to_tensor(img)  # [C,H,W], float32 in [0,1]
            loaded.append((p, tensor))
        except Exception as e:
            print(f"[WARN] Failed to load {p}: {e}", file=sys.stderr)

    if len(loaded) == 0:
        raise RuntimeError("모든 이미지 로드에 실패했습니다; 시각화할 이미지가 없습니다.")

    return loaded


# 실제 실행
images = load_images(all_image_paths)
print(f"[INFO] Successfully loaded {len(images)} images")

# 첫 1개 확인
images[0][0], images[0][1].shape

[INFO] Successfully loaded 1280 images


(PosixPath('/home/jener05458/src/EdgeMI/TBD_MI/dataset/deit_base_16_imagenet/SMI_iter/W8A8/SMI-100-[50, 100, 200, 300]-[0.3, 0.3, 0.3, 0.3]-0-32-W8A8/0-0.png'),
 torch.Size([3, 224, 224]))

In [49]:
# Cell 4: 폴더 이름에서 iteration 숫자 추출

# 예: "DMI-100-0-32-W4A8", "SMI-100-[50,100,200,300]-[...]"
_iter_pattern = re.compile(r"^(DMI|SMI)-(\d+)-")

def extract_iter_from_path(path: Path) -> int:
    """
    이미지 파일의 상위 디렉토리 이름에서 iteration 숫자를 추출.
    예:
      - DMI-100-0-32-W4A8 -> 100
      - SMI-400-[50,100,200,300]-[...] -> 400
    """
    folder_name = path.parent.name
    m = _iter_pattern.match(folder_name)
    if m is None:
        raise ValueError(
            f"폴더 이름에서 iter를 찾을 수 없습니다: '{folder_name}' (path={path})"
        )
    return int(m.group(2))


# ---- 간단 테스트 ----
print("[TEST] 첫 5개 이미지에 대해 폴더/iter 확인")

for i in range(min(5, len(images))):
    p = images[i][0]
    it = extract_iter_from_path(p)
    print(f"{p.parent.name:30s} -> iter = {it}")


[TEST] 첫 5개 이미지에 대해 폴더/iter 확인
SMI-100-[50, 100, 200, 300]-[0.3, 0.3, 0.3, 0.3]-0-32-W8A8 -> iter = 100
SMI-100-[50, 100, 200, 300]-[0.3, 0.3, 0.3, 0.3]-0-32-W8A8 -> iter = 100
SMI-100-[50, 100, 200, 300]-[0.3, 0.3, 0.3, 0.3]-0-32-W8A8 -> iter = 100
SMI-100-[50, 100, 200, 300]-[0.3, 0.3, 0.3, 0.3]-0-32-W8A8 -> iter = 100
SMI-100-[50, 100, 200, 300]-[0.3, 0.3, 0.3, 0.3]-0-32-W8A8 -> iter = 100


In [50]:
# Cell 5: 수정된 high-frequency 에너지 비율 계산 (정확한 원형 annulus만 사용)

def compute_high_freq_ratio(image_tensor: torch.Tensor, cutoff_ratio: float = 0.7) -> float:
    """
    image_tensor: [C,H,W], 0~1 float
    cutoff_ratio: 0~1 사이 값
      - 0.7이면: 내접원 반지름의 70% 바깥을 high-frequency로 계산
    """

    # 1) RGB -> grayscale
    img_np = image_tensor.numpy()          # (C,H,W)
    gray = img_np.mean(axis=0)             # (H,W)

    H, W = gray.shape

    # 2) FFT
    fft = np.fft.fft2(gray)
    fft_shift = np.fft.fftshift(fft)
    mag = np.abs(fft_shift)                # magnitude spectrum

    # 3) radius grid
    y, x = np.indices((H, W))
    cy, cx = (H - 1) / 2.0, (W - 1) / 2.0
    r = np.sqrt((x - cx)**2 + (y - cy)**2)

    # 4) 내접원의 최대 반지름 (corner 문제 제거)
    R_max = min(cy, H-1-cy, cx, W-1-cx)

    # cutoff 계산
    cutoff_radius = cutoff_ratio * R_max

    # 5) annulus mask (원형 도넛)
    high_mask = (r >= cutoff_radius) & (r <= R_max)

    # 6) 에너지 계산
    total_energy = mag.sum()
    high_energy = mag[high_mask].sum()

    eps = 1e-8
    return float(high_energy / (total_energy + eps))


# 테스트
compute_high_freq_ratio(images[0][1], cutoff_ratio=CUTOFF_RATIO)


0.1704330944317057

In [51]:
# -----------------------------
# 그래프 저장 (mean curve only)
# -----------------------------
plt.figure(figsize=(8, 5))
plt.plot(
    iters_np,
    mean_np,
    "-o",
    linewidth=2,
    markersize=6,
)
plt.xlabel("Iteration")
plt.ylabel("High-frequency energy ratio")
plt.title(f"Iteration vs High-frequency ratio (cutoff={CUTOFF_RATIO})")
plt.grid(True)

fig_path = SAVE_DIR / "iteration_vs_highfreq.png"
plt.tight_layout()
plt.savefig(fig_path)
plt.close()

print(f"[MODE1] 플롯 저장: {fig_path}")
print("[MODE1] 완료!")


[MODE1] 플롯 저장: /home/jener05458/src/EdgeMI/TBD_MI/observation/iter_noise/iteration_vs_highfreq.png
[MODE1] 완료!
